In [1]:
from moztelemetry.dataset import Dataset
import json

dataset = Dataset.from_source('telemetry')

dataset = (dataset.where(docType='OTHER')
                  .where(appName='Firefox')
                  .where(appUpdateChannel='beta')
                  .where(submissionDate=lambda x: x >= '20170712'))

records = dataset.records(sc)

logs = records.filter(lambda x: x["meta"]["docType"] == "tls13-middlebox-beta")

print logs.count()

beta_logs = logs.take(1000000)


fetching 92.89840MB in 9924 files...
659825


In [2]:
with open('beta-logs.json', 'w') as f:
    for l in beta_logs:
        print >> f, json.dumps(l)


<class 'pyspark.rdd.PipelinedRDD'>


In [39]:
import sys
import traceback

def intToHex(num):
    return hex(num) if num is not None else None

def getErrorString(ec):
    if ec in error_names:
        error_names[ec]
        
    return []

error_names = {}

with open("codes.txt", "r") as f:
    for line in f:
        tokens = line.strip().split()
        
        if int(tokens[0]) not in error_names:
            error_names[int(tokens[0])] = []
            
        error_names[int(tokens[0])].append(tokens[1])

security_states = set()
status = set()
errorCodes = set()
status_error_codes = set()

try:
    with open("beta-logs-finihsed.json", "r") as f:
        for line in f:
            data = json.loads(line.strip())

            for test in data["payload"]["tests"]:
                if "securityState" in test["result"]:
                    security_states.add(test["result"]["securityState"])
                
                s = (test["result"]["status"] if "status" in test["result"] else None)
                status.add(s)
                
                ec = (test["result"]["errorCode"] if "errorCode" in test["result"] else None)
                errorCodes.add(ec)
                
                status_error_codes.add((s, ec))
except:
    print >> sys.stderr, traceback.format_exc()
    print >> sys.stderr, json.dumps(data, indent=4, separators=(',', ': '))

print "securityState: ", [intToHex(x) for x in security_states]
print "status: ", [intToHex(x) for x in status]
print "errorCode: ", [intToHex(x) for x in errorCodes]

print "status errorCode pair: "

sorted_ = sorted([(x, y) for x, y in status_error_codes], key=lambda z: (z[0], z[1]))

for x, y in sorted_:
    print "%s (%s)\t%s (%s)" % (intToHex(x), getErrorString(x), intToHex(y), getErrorString(y))


securityState:  ['0x40002', '0x4']
status:  ['0x0', '0x804b0002', '0x80004004', '0x80004005', '0x804b000d', '0x804b000e', '0x80040111', '0x804b0014', '0x805a2fef', '0x805a2f9c', '0x804b001e', '0x805a2fa1', '0x805a2fa2', None, '0x805a1f76', '0x804b0047', '0x804b0048', '0x805a2fca', '0x805a1f50', '0x805a1fe2', '0x805a2fe7', '0x805a3ffd', '0x805a2ff1', '0x805a1ff3', '0x805a2ff4', '0x805a1ff6', '0x805a2ff9', '0x805a3ffa', '0x805a1f7c', '0x805a1f7d', '0x805a2ffe', '0x805a1fff']
errorCode:  ['0x0', '-0x1fff', '-0x2ffe', '-0x1f7d', '-0x1f7c', '-0x3ffa', '-0x2ff9', '-0x1ff6', '-0x2ff4', '-0x1ff3', '-0x2ff1', '-0x2ff0', '-0x2fef', '-0x3ffd', '-0x2fe7', '-0x1fe2', '-0x1f50', None, '-0x2fca', '-0x1749', '-0x2fc5', '-0x1f76', '-0x1732', '-0x1728', '-0x2fa2', '-0x2fa1', '-0x2f9c']
status errorCode pair: 


[(None, None),
 ('0x0', None),
 ('0x0', '0x0'),
 ('0x80004004', None),
 ('0x80004004', '-0x1728'),
 ('0x80004004', '-0x1732'),
 ('0x80004004', '-0x1749'),
 ('0x80004004', '-0x2fc5'),
 ('0x80004004', '-0x2ff0'),
 ('0x80004004', '-0x2ff1'),
 ('0x80004004', '0x0'),
 ('0x80004005', None),
 ('0x80004005', '0x0'),
 ('0x80040111', None),
 ('0x804b0002', None),
 ('0x804b0002', '0x0'),
 ('0x804b000d', None),
 ('0x804b000e', None),
 ('0x804b0014', None),
 ('0x804b0014', '-0x1728'),
 ('0x804b0014', '-0x1749'),
 ('0x804b0014', '0x0'),
 ('0x804b001e', None),
 ('0x804b0047', '-0x1732'),
 ('0x804b0048', None),
 ('0x805a1f50', '-0x1f50'),
 ('0x805a1f76', '-0x1f76'),
 ('0x805a1f7c', '-0x1f7c'),
 ('0x805a1f7d', '-0x1f7d'),
 ('0x805a1fe2', '-0x1fe2'),
 ('0x805a1ff3', '-0x1ff3'),
 ('0x805a1ff6', '-0x1ff6'),
 ('0x805a1fff', '-0x1fff'),
 ('0x805a2f9c', '-0x2f9c'),
 ('0x805a2fa1', '-0x2fa1'),
 ('0x805a2fa2', '-0x2fa2'),
 ('0x805a2fca', '-0x2fca'),
 ('0x805a2fe7', '-0x2fe7'),
 ('0x805a2fef', '-0x2fef'),
 ('0x